In [1]:
from langchain.llms import GooglePalm

api_key='AJzaSyMkmYcLNvR_SZ0XtQ2hiqVF1mHdN7Z-J_j'#(Not the Real Key)

llm = GooglePalm(google_api_key=api_key, temperature=0.1)

In [3]:
poem = llm("Write a 4 line poem for Rose")
print(poem)

**Rose**

A rose is a flower,
A symbol of love and beauty.
It blooms in the spring,
And brings joy to all who see it.


In [4]:
from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

In [ ]:
#%pip install --upgrade langchain langchain-google-vertexai

In [5]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='codebasics_faqs.csv', source_column="prompt")

# Store the loaded data in the 'data' variable
data = loader.load()

- Now to select embeddings you can use anyone from the Langchain Embeddings Documentation and se its implementation 
- Also to see its effectiveness you can use cosine similary 

In [6]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("What is your refund policy?")

load INSTRUCTOR_Transformer
max_seq_length  512


In [7]:
print(e)

[-0.0438980758190155, 0.007685545366257429, -0.009231900796294212, 0.024496251717209816, 0.03359227254986763, 0.06253515928983688, 0.010531389154493809, 0.02161242812871933, -0.022162247449159622, 0.029747508466243744, 0.044324278831481934, -0.014968828298151493, 0.046558331698179245, 0.03853654861450195, -0.06129590421915054, -0.04503975436091423, -0.07021936029195786, -0.014154857955873013, -0.026523813605308533, 0.012313468381762505, 0.06716736406087875, -0.005974954925477505, -0.01128630805760622, 0.01893932744860649, 0.011767148040235043, 0.019715406000614166, -0.0245383158326149, 0.02754220925271511, 0.06479945778846741, -0.05754319950938225, 0.01590232364833355, -0.031119760125875473, -0.057875435799360275, -0.04131997749209404, -0.024895019829273224, 0.026121459901332855, 0.014952125027775764, 0.00224844622425735, 0.03822099789977074, 0.013710802420973778, -0.004282941576093435, 0.02549811452627182, 0.04529273509979248, -0.04229624941945076, 0.0005462196422740817, -0.0363113433

In [8]:
# Checkng the Cosine-similary to see its accuracy 
# Making the list into a Numpy Array and then Reshaping 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming instructor_embeddings.embed_query returns lists of embeddings
sentences_a = "The Federal Reserve on Wednesday raised its benchmark interest rate."
sentences_b = "The funds rose less than 0.5 per cent on Friday"
embeddings_a = np.array(instructor_embeddings.embed_query(sentences_a)).reshape(1, -1)
embeddings_b = np.array(instructor_embeddings.embed_query(sentences_b)).reshape(1, -1)

similarities = cosine_similarity(embeddings_a, embeddings_b)
print(similarities)


[[0.81030708]]


In [9]:
print(len(e))
e[:5]


768


[-0.0438980758190155,
 0.007685545366257429,
 -0.009231900796294212,
 0.024496251717209816,
 0.03359227254986763]

In [10]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)
# The score_threshold is used to check that the cosine similarity should be more than 70% 

What retriver object does is it compares the given Input's embeddings to the the embedding in the Vector Database

In [11]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

- Now here we use prompt template code to specifically the kind on input we want 
- PrompTtemplate is a datatype with parameters 

In [12]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [13]:
chain('Do you provide guarantee for a job?')

{'query': 'Do you provide guarantee for a job?',
 'result': 'The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Will this bootcamp guarantee me a job?\nresponse: The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this b